In [1]:
import pandas as pd
from functools import reduce
import numpy as np
import warnings
warnings.filterwarnings('ignore')

allstar = pd.read_csv('AllstarFull.csv', encoding='latin1')
appearances = pd.read_csv('Appearances.csv', encoding='latin1')
awards = pd.read_csv('AwardsPlayers.csv', encoding='latin1')
awardsshare = pd.read_csv('AwardsSharePlayers.csv', encoding='latin1')
batting = pd.read_csv('Batting.csv', encoding='latin1')
halloffame = pd.read_csv('HallOfFame.csv', encoding='latin1')
people = pd.read_csv('People.csv', encoding='latin1')
teams = pd.read_csv('Teams.csv', encoding='latin1')

C:\Users\student\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
position_columns = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']
appearances['gpbyappearance'] = appearances[position_columns].sum(axis=1)
appearances = appearances[appearances['gpbyappearance'] > 0]

In [3]:
current_df = people
hall_of_fame_df = halloffame
inducted_hof_df = hall_of_fame_df[hall_of_fame_df['inducted'] == 'Y']
inducted_hof_df = inducted_hof_df.drop_duplicates(subset=['playerID'])
inducted_hof_df = inducted_hof_df[inducted_hof_df['category'] == 'Player']
merged_df = pd.merge(current_df, inducted_hof_df[['playerID', 'inducted']], on='playerID', how='left')
merged_df['inducted'] = merged_df['inducted'].fillna('N')
merged_df['inducted'] = merged_df['inducted'].replace({'Y': 'HOF', 'N': 'Non-HOF'})

In [4]:
mergedf1 = [batting, appearances]
mergeddf1 = reduce(lambda left, right: pd.merge(left, right, on=['playerID', 'yearID', 'teamID', 'lgID'], how='left'), mergedf1)

In [5]:
mergeddf11 = mergeddf1[((mergeddf1['AB'] + mergeddf1['BB'] + mergeddf1['HBP'] + mergeddf1['SH'] + mergeddf1['SF']) / 162 >= 3.1) & (mergeddf1['yearID'] != 2020)]
mergeddf12 = mergeddf1[((mergeddf1['AB'] + mergeddf1['BB'] + mergeddf1['HBP'] + mergeddf1['SH'] + mergeddf1['SF']) / 60 >= 3.1) & (mergeddf1['yearID'] == 2020)]
mergeddf_final = pd.concat([mergeddf11, mergeddf12])
mergeddf1 = mergeddf_final
pd.set_option('display.max_columns', None)

In [6]:
position_columns = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']
mergeddf1['primaryposition'] = mergeddf1[position_columns].idxmax(axis=1)

In [7]:
position_mapping = {'G_p': 'Pitcher', 'G_c': 'Catcher', 'G_1b': 'First Base', 'G_2b': 'Second Base', 'G_3b': 'Third Base',
    'G_ss': 'Shortstop', 'G_lf': 'Left Field', 'G_cf': 'Center Field', 'G_rf': 'Right Field', 'G_dh': 'Designated Hitter'}
mergeddf1['primaryposition'] = mergeddf1['primaryposition'].replace(position_mapping)
mergeddf1 = mergeddf1[mergeddf1['primaryposition'] != 'Pitcher']

In [8]:
scolumns1 = ['playerID', 'yearID', 'teamID', 'lgID', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO',
             'IBB', 'HBP', 'SH', 'SF', 'GIDP', 'primaryposition']
mdf1new = mergeddf1[scolumns1]

In [9]:
mergedf3 = [mdf1new, merged_df]
mergeddf3 = reduce(lambda left, right: pd.merge(left, right, on=['playerID'], how='left'), mergedf3)
mergedf3 = [mergeddf3, awards, allstar]
mergeddf3 = reduce(lambda left, right: pd.merge(left, right, on=['playerID', 'yearID'], how='left'), mergedf3)

In [10]:
mergeddf3['gameID'] = mergeddf3['gameID'].fillna("Not an All Star")
mergeddf3['gameID'] = mergeddf3['gameID'].apply(lambda x: 'All Star' if x != "Not an All Star" else x)

In [11]:
mergeddf3 = mergeddf3.rename(columns={'lgID_x': 'lgID'})
mergeddf3 = mergeddf3.rename(columns={'teamID_x': 'teamID'})
mergeddf3 = mergeddf3.rename(columns={'gameID': 'as'})

In [12]:
scolumns3 = ['playerID', 'yearID', 'teamID', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB',
             'HBP', 'SH', 'SF', 'GIDP', 'primaryposition', 'birthCountry', 'birthState', 'nameFirst', 'nameLast', 'weight',
             'height', 'bats', 'throws', 'as', 'debut', 'finalGame', 'inducted', 'awardID']
mdf3new = mergeddf3[scolumns3]

In [13]:
mdf3new['nameFirst'] = mdf3new['nameFirst'].fillna('')
mdf3new['nameLast'] = mdf3new['nameLast'].fillna('')
mdf3new['awardID'] = mdf3new['awardID'].fillna('No Awards')
mdf3new['awardID'] = mdf3new['awardID'].replace('SIlver Slugger', 'Silver Slugger')
mdf3new.loc[:, 'playername'] = mdf3new['nameFirst'] + ' ' + mdf3new['nameLast']
mdf3new['BA'] = round(mdf3new['H']/mdf3new['AB'],3)
mdf3new['BA'] = mdf3new['BA'].fillna(0)
mdf3new['birthState'] = mdf3new['birthState'].apply(lambda x: x if pd.notna(x) else 'Unknown')

In [14]:
mdf3new['as'] = mdf3new['as'].replace({'All Star': 1, 'Not an All Star': 0})
mdf3new['poty'] = mdf3new['awardID'].apply(lambda x: 1 if 'TSN Player of the Year' in x else 0)
mdf3new['mvp'] = mdf3new['awardID'].apply(lambda x: 1 if 'Most Valuable Player' in x else 0)
mdf3new['gg'] = mdf3new['awardID'].apply(lambda x: 1 if 'Gold Glove' in x else 0)
mdf3new['ss'] = mdf3new['awardID'].apply(lambda x: 1 if 'Silver Slugger' in x else 0)
mdf3new['roty'] = mdf3new['awardID'].apply(lambda x: 1 if 'Rookie of the Year' in x else 0)
mdf3new['mlpoty'] = mdf3new['awardID'].apply(lambda x: 1 if 'TSN Major League Player of the Year' in x else 0)
mdf3new['ha'] = mdf3new['awardID'].apply(lambda x: 1 if 'Hank Aaron Award' in x else 0)
mdf3new['br'] = mdf3new['awardID'].apply(lambda x: 1 if 'Babe Ruth Award' in x else 0)
mdf3new['pg'] = mdf3new['awardID'].apply(lambda x: 1 if 'Platinum Glove' in x else 0)
mdf3new['alcsmvp'] = mdf3new['awardID'].apply(lambda x: 1 if 'ALCS MVP' in x else 0)
mdf3new['nlcsmvp'] = mdf3new['awardID'].apply(lambda x: 1 if 'NLCS MVP' in x else 0)
mdf3new['wsmvp'] = mdf3new['awardID'].apply(lambda x: 1 if 'World Series MVP' in x else 0)
mdf3new['totalawards'] = mdf3new['poty']+mdf3new['mvp']+mdf3new['gg']+mdf3new['ss']+mdf3new['roty']+mdf3new['mlpoty']+mdf3new['ha']+mdf3new['br']+mdf3new['pg']+mdf3new['alcsmvp']+mdf3new['nlcsmvp']+mdf3new['wsmvp']

In [15]:
mdf3new.drop(columns=['nameFirst', 'nameLast', 'awardID'], inplace=True)

In [16]:
mdf3new = mdf3new.dropna(subset=['primaryposition'])
mdf3new = mdf3new.dropna(subset=['CS'])
mdf3new = mdf3new.dropna(subset=['GIDP'])
mdf3new = mdf3new.dropna(subset=['IBB'])
mdf3new = mdf3new.dropna(subset=['SF'])
mdf3new = mdf3new.dropna(subset=['SH'])
mdf3new = mdf3new.dropna(subset=['debut'])
mdf3new = mdf3new.dropna(subset=['throws'])
mdf3new = mdf3new.dropna(subset=['bats'])

In [17]:
mergedf4 = [mdf3new, teams]
mergeddf4 = reduce(lambda left, right: pd.merge(left, right, on=['yearID', 'teamID'], how='left'), mergedf4)

In [18]:
scolumns4 = ['playerID', 'yearID', 'teamID', 'G_x', 'AB_x', 'R_x', 'H_x',
       '2B_x', '3B_x', 'HR_x', 'RBI', 'SB_x', 'CS_x', 'BB_x', 'SO_x', 'IBB',
       'HBP_x', 'SH', 'SF_x', 'GIDP', 'primaryposition',
       'birthCountry', 'birthState', 'weight', 'height', 'bats', 'throws',
       'playername', 'BA', 'name', 'debut', 'finalGame', 'inducted', 'as', 'poty', 'mvp', 'gg', 'ss', 'roty', 'mlpoty',
       'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp', 'totalawards']
mdf4new = mergeddf4[scolumns4]

In [19]:
country_mapping = {'USA': 'United States', 'P.R.': 'Puerto Rico', 'D.R.': 'Dominican Republic', 'CAN': 'Canada',
    'V.I.': 'Virgin Islands'}
mdf4new['birthCountry'] = mdf4new['birthCountry'].replace(country_mapping)

In [20]:
bats_mapping = {'L': 'Left', 'R': 'Right', 'B': 'Switch'}
mdf4new['bats'] = mdf4new['bats'].replace(bats_mapping)

In [21]:
throws_mapping = {'L': 'Left', 'R': 'Right'}
mdf4new['throws'] = mdf4new['throws'].replace(throws_mapping)

In [22]:
mdf4new['debut'] = pd.to_datetime(mdf4new['debut'])
mdf4new['debutyear'] = mdf4new['debut'].dt.year
mdf4new['finalGame'] = pd.to_datetime(mdf4new['finalGame'])
mdf4new['finalyear'] = mdf4new['finalGame'].dt.year
mdf4new['currentcareerlength'] = mdf4new['yearID'] - mdf4new['debutyear']

In [23]:
mdf4new = mdf4new.rename(columns={'playerID': 'id', 'yearID': 'year', 'teamID': 'teamid', 'lgID_x': 'league', 'G_x': 'games',
                               'AB_x': 'atbats', 'R_x': 'runs', 'H_x': 'hits', '2B_x': 'doubles', '3B_x': 'triples',
                               'HR_x': 'homeruns', 'RBI': 'runsbattedin', 'SB_x': 'stolenbases', 'CS_x': 'caughtstealing',
                               'BB_x': 'walks', 'SO_x': 'strikeouts', 'IBB': 'intentionalwalks', 'HBP_x': 'hitbypitch',
                               'SH': 'sacrificehits', 'SF_x': 'sacrificeflys', 'GIDP': 'groundedintodoubleplay', 'awardID': 'award',
                               'birthCountry': 'country', 'birthState': 'statecity', 'BA': 'battingaverage', 'name': 'teamname'})

In [24]:
mdf4new.drop(columns=['teamid'], inplace=True)

In [25]:
mdf4columnorder = ['id', 'playername', 'year', 'teamname', 'games', 'battingaverage', 'atbats', 'hits', 'doubles', 'triples',
                   'homeruns', 'runsbattedin', 'runs', 'stolenbases', 'caughtstealing', 'strikeouts', 'walks', 'intentionalwalks',
                   'hitbypitch', 'sacrificehits', 'sacrificeflys', 'groundedintodoubleplay', 'primaryposition', 'country',
                   'statecity', 'height', 'weight', 'bats', 'throws', 'as', 'poty', 'mvp', 'gg', 'ss', 'roty', 'mlpoty',
       'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp', 'totalawards', 'currentcareerlength', 'debutyear', 'finalyear', 'inducted']
mdf4new = mdf4new[mdf4columnorder]
mdf4new = mdf4new.drop_duplicates()

In [26]:
data = mdf4new.sort_values(by=['id', 'year'])

In [27]:
data = data[data['currentcareerlength'] <= 5]

In [28]:
df_awards = data.groupby(['id', 'playername', 'year']).agg({
    'as': 'sum',
    'poty': 'sum',
    'mvp': 'sum',
    'gg': 'sum',
    'ss': 'sum',
    'roty': 'sum',
    'mlpoty': 'sum',
    'ha': 'sum',
    'br': 'sum',
    'pg': 'sum',
    'alcsmvp': 'sum',
    'nlcsmvp': 'sum',
    'wsmvp': 'sum',
}).reset_index()
awards_columns = ['poty', 'mvp', 'gg', 'ss', 'roty', 'mlpoty', 'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp']
df_awards['totalawards'] = df_awards[awards_columns].sum(axis=1)
df_awards['as'] = df_awards['as'].apply(lambda x: 1 if x > 1 else x)

In [29]:
data = data.drop(['as', 'poty', 'mvp', 'gg', 'ss', 'roty', 'mlpoty', 'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp', 'totalawards'], axis=1)

In [30]:
test = [data, df_awards]
final_df = reduce(lambda left, right: pd.merge(left, right, on=['id', 'year'], how='left'), test)

In [31]:
final_df.head(10)
final_df.drop_duplicates()
final_df = final_df.drop_duplicates(subset=['id', 'year'])

In [32]:
aggregated_data = final_df.groupby('id').agg({
    'hits': 'mean',
    'homeruns': 'mean',
    'atbats': 'mean',
    'games': 'mean',
    'doubles': 'mean',
    'triples': 'mean',
    'runsbattedin': 'mean',
    'runs': 'mean',
    'stolenbases': 'mean',
    'caughtstealing': 'mean',
    'strikeouts': 'mean',
    'walks': 'mean',
    'intentionalwalks': 'mean',
    'hitbypitch': 'mean',
    'sacrificehits': 'mean',
    'sacrificeflys': 'mean',
    'groundedintodoubleplay': 'mean',
    'as': 'sum',
    'poty': 'sum',
    'mvp': 'sum',
    'gg': 'sum',
    'ss': 'sum',
    'roty': 'sum',
    'mlpoty': 'sum',
    'ha': 'sum',
    'br': 'sum',
    'pg': 'sum',
    'alcsmvp': 'sum',
    'nlcsmvp': 'sum',
    'wsmvp': 'sum',
    'totalawards': 'sum'}).reset_index()

In [33]:
final_data = pd.merge(aggregated_data, data, on='id', how='left')

In [34]:
finalcolumns = ['id', 'hits_x', 'homeruns_x', 'atbats_x', 'games_x', 'doubles_x',
       'triples_x', 'runsbattedin_x', 'runs_x', 'stolenbases_x',
       'caughtstealing_x', 'strikeouts_x', 'walks_x', 'intentionalwalks_x',
       'hitbypitch_x', 'sacrificehits_x', 'sacrificeflys_x',
       'groundedintodoubleplay_x', 'playername', 'country', 'statecity',
       'height', 'weight', 'bats', 'throws', 'as', 'poty', 'mvp', 'gg', 'ss',
       'roty', 'mlpoty', 'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp',
       'totalawards', 'debutyear', 'finalyear',
       'inducted']

In [35]:
final_data = final_data[finalcolumns]

In [36]:
final_data = final_data.rename(columns={'hits_x': 'hits', 'homeruns_x': 'homeruns', 'atbats_x': 'atbats', 'games_x': 'games',
                                        'doubles_x': 'doubles', 'triples_x': 'triples', 'runsbattedin_x': 'runsbattedin',
                                        'runs_x': 'runs', 'stolenbases_x': 'stolenbases', 'caughtstealing_x': 'caughtstealing',
                                        'strikeouts_x': 'strikeouts', 'walks_x': 'walks', 'intentionalwalks_x': 'intentionalwalks',
                                        'hitbypitch_x': 'hitbypitch', 'sacrificehits_x': 'sacrificehits',
                                        'sacrificeflys_x': 'sacrificeflys', 'groundedintodoubleplay_x': 'groundedintodoubleplay',
                                        'as_x': 'as', 'poty_x': 'poty', 'mvp_x': 'mvp', 'gg_x': 'gg', 'ss_x': 'ss', 'roty_x': 'roty',
                                        'mlpoty_x': 'mlpoty', 'ha_x': 'ha', 'br_x': 'br', 'pg_x': 'pg', 'alcsmvp_x': 'alcsmvp',
                                        'nlcsmvp_x': 'nlcsmvp','totalawards_x': 'totalawards'})

In [37]:
final_data = final_data.drop_duplicates()

In [38]:
final_data['battingaverage'] = round(final_data['hits']/final_data['atbats'],3)
final_data['battingaverage'] = final_data['battingaverage'].fillna(0)
final_data['onbasepercent'] = round((final_data['hits']+final_data['walks']+final_data['hitbypitch'])/(final_data['atbats']+final_data['walks']+final_data['hitbypitch']+final_data['sacrificeflys']),3)
final_data['onbasepercent'] = final_data['onbasepercent'].fillna(0)
final_data['extrabasehits'] = final_data['doubles']+final_data['triples']+final_data['homeruns']
final_data['extrabasehits'] = final_data['extrabasehits'].fillna(0)
final_data['totalbases'] = (final_data['hits']-final_data['doubles']-final_data['triples']-final_data['homeruns'])+(2*final_data['doubles'])+(3*final_data['triples'])+(4*final_data['homeruns'])
final_data['totalbases'] = final_data['totalbases'].fillna(0)
final_data['slugging'] = round(final_data['totalbases']/final_data['atbats'],3)
final_data['slugging'] = final_data['slugging'].fillna(0)
final_data['onbaseslugging'] = round(final_data['onbasepercent']+final_data['slugging'],3)
final_data['onbaseslugging'] = final_data['onbaseslugging'].fillna(0)

In [39]:
finalcolumnorder = ['id', 'games', 'atbats', 'battingaverage', 'onbasepercent', 'slugging', 'onbaseslugging', 'hits', 'doubles',
                    'triples', 'homeruns', 'extrabasehits', 'totalbases', 'runsbattedin', 'runs', 'stolenbases', 'caughtstealing',
                    'strikeouts', 'walks', 'intentionalwalks', 'hitbypitch', 'sacrificehits', 'sacrificeflys',
                    'groundedintodoubleplay', 'playername', 'country', 'statecity', 'height', 'weight', 'bats', 'throws', 'as', 'poty',
                    'mvp', 'gg', 'ss', 'roty', 'mlpoty', 'ha', 'br', 'pg', 'alcsmvp', 'nlcsmvp', 'wsmvp', 'totalawards', 'debutyear', 'finalyear',
                    'inducted']
final_data = final_data[finalcolumnorder]

In [40]:
final_data

,id,games,atbats,battingaverage,onbasepercent,slugging,onbaseslugging,hits,doubles,triples,homeruns,extrabasehits,totalbases,runsbattedin,runs,stolenbases,caughtstealing,strikeouts,walks,intentionalwalks,hitbypitch,sacrificehits,sacrificeflys,groundedintodoubleplay,playername,country,statecity,height,weight,bats,throws,as,poty,mvp,gg,ss,roty,mlpoty,ha,br,pg,alcsmvp,nlcsmvp,wsmvp,totalawards,debutyear,finalyear,inducted
0,aaronha01,152.800000,611.200000,0.329,0.379,0.577,0.956,201.200000,35.600000,8.000000,33.200000,76.800000,352.400000,109.600000,110.800000,3.600000,1.400000,55.200000,50.600000,11.800000,2.000000,2.400000,5.200000,18.800000,Hank Aaron,United States,AL,72.0,180.0,Right,Right,5,1,1,2,0,0,0,0,0,0,0,0,0,4,1954,1976,HOF
8,abernbr01,117.000000,463.000000,0.242,0.288,0.311,0.599,112.000000,18.000000,4.000000,2.000000,24.000000,144.000000,40.000000,46.000000,10.000000,4.000000,46.000000,25.000000,0.000000,6.000000,8.000000,2.000000,8.000000,Brent Abernathy,United States,GA,73.0,185.0,Right,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2001,2005,Non-HOF
9,abramcj01,151.000000,563.000000,0.245,0.300,0.412,0.712,138.000000,28.000000,6.000000,18.000000,52.000000,232.000000,64.000000,83.000000,47.000000,4.000000,118.000000,32.000000,2.000000,13.000000,3.000000,3.000000,7.000000,CJ Abrams,United States,GA,74.0,185.0,Left,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,2022,Non-HOF
10,abreubo01,154.750000,551.750000,0.313,0.416,0.537,0.953,172.500000,38.500000,7.750000,23.250000,69.500000,296.250000,89.000000,101.750000,27.500000,10.250000,124.750000,99.750000,10.500000,1.250000,1.000000,5.000000,11.000000,Bobby Abreu,Venezuela,Aragua,72.0,220.0,Left,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1996,2014,Non-HOF
14,abreujo02,150.166667,591.166667,0.293,0.349,0.513,0.862,173.000000,36.333333,2.333333,29.833333,68.500000,303.500000,101.833333,80.500000,1.666667,0.833333,129.333333,40.833333,8.333333,13.333333,0.000000,5.666667,18.333333,Jose Abreu,Cuba,Cienfuegos,75.0,235.0,Right,Right,3,0,0,0,2,1,0,0,0,0,0,0,0,3,2014,2022,Non-HOF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,zauchno01,130.000000,477.000000,0.239,0.335,0.430,0.765,114.000000,10.000000,0.000000,27.000000,37.000000,205.000000,93.000000,65.000000,3.000000,0.000000,105.000000,69.000000,1.000000,3.000000,1.000000,6.000000,8.000000,Norm Zauchin,United States,MI,76.0,220.0,Right,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1951,1959,Non-HOF
4439,zeileto01,145.500000,517.500000,0.266,0.348,0.404,0.752,137.500000,28.750000,2.750000,12.500000,44.000000,209.250000,72.250000,67.750000,7.750000,7.250000,79.250000,66.750000,3.750000,1.750000,0.000000,6.250000,13.000000,Todd Zeile,United States,CA,73.0,190.0,Right,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1989,2004,Non-HOF
4443,zimmery01,154.500000,600.500000,0.287,0.357,0.490,0.847,172.250000,39.750000,2.750000,25.500000,68.000000,294.000000,98.000000,94.500000,5.250000,2.500000,115.500000,65.750000,6.250000,2.750000,0.250000,5.750000,19.750000,Ryan Zimmerman,United States,NC,75.0,215.0,Right,Right,1,0,0,1,2,0,0,0,0,0,0,0,0,3,2005,2021,Non-HOF
4448,ziskri01,150.333333,540.333333,0.297,0.367,0.471,0.838,160.666667,30.666667,2.666667,19.333333,52.666667,254.666667,88.000000,78.333333,0.666667,0.666667,98.666667,61.666667,6.333333,0.666667,0.000000,5.000000,14.666667,Richie Zisk,United States,NY,73.0,200.0,Right,Right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1971,1983,Non-HOF


In [41]:
final_data.to_csv('Q5yravg_HOF.csv', index=False)